#### Step 1 : Load the workspace configurations

In [1]:
from azureml.core import Workspace, Dataset, Experiment

subscription_id = 'ba04a809-bb92-41ca-9b73-b1baa9000986'
resource_group = 'MLOPS'
workspace_name = 'Machine_Learning_Projects'

workspace = Workspace(subscription_id, resource_group, workspace_name)

#### Step 2 : Start with the experiment 

In [2]:
experiment = Experiment(workspace=workspace, name="iris")
dataset = Dataset.get_by_name(workspace=workspace, name="Iris")

df = dataset.to_pandas_dataframe()
df.head(5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


#### Step 3 : Create a Decision Tree Classifier

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

X = df[['sepal_length','sepal_width','petal_length','petal_width']]
Y = df['species']

run = experiment.start_logging()
test_size = 0.40

run.log("train_test_split", test_size)
run.log("Type of Algorithm", "Decision Tree")

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size, shuffle=True)

# step 1 : initialize the model class
clf = DecisionTreeClassifier(criterion="gini", max_depth=4)

# step 2 : train the model with training data
clf.fit(X_train, y_train)

# step 3 : evaluate the model with testing data
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

run.log("accuracy",accuracy*100)

# pickling the file 

model_name = "iris_decision_tree.pkl"
filepath = "outputs/"+model_name 

joblib.dump(value= clf, filename=filepath)
run.upload_file(name=model_name, path_or_stream=filepath)
run.complete()

TypeError: Singleton array array(0.4) cannot be considered a valid collection.

#### Step 4 : Register the model

In [ ]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
import sklearn

In [9]:
model = Model.register(
    workspace=workspace,
    model_name = "intellipat",
    model_path = filepath,
    model_framework= Model.Framework.SCIKITLEARN,
    model_framework_version = sklearn.__version__,
    description = "This is a sample DT classifier",
    resource_configuration = ResourceConfiguration(cpu=1, memory_in_gb=0.5)
)

print("Name:", model.name)
print("version:", model.version)

Registering model intellipat
Name: intellipat
version: 2


#### Step 5 : Deploying the model